<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.1/533.1 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=fb7e01dfa0cdc8e798ee597877b58afe47372f3f3757ba7937cdc375c5311a70
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-08 09:25:53
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.43 C
-------------------
Today PnL: -16.39 K (-0.11%)
Current PnL: -21.78 L (-14.27%)
CY Booked + Current PnL: -9.13 L (-5.98%)
-------------------
Total profit:  2.05 L
Total loss:  -23.83 L
-------------------
Total Booked + Current PnL: 17.85 L (14.21%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.86%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 88.77 L (62.17%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.48%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-8.03,60.0,X-LC,6.42,230568.0,20202.0,8577.0,-1.21,9.60,3.72,13.68,37.0,2.36,1.63,27.71,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-3.88,68.0,M-LC,11.83,174302.0,33462.0,11033.0,2.10,23.76,6.33,31.59,66.0,3.03,1.23,56.54,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,103.20,53.0,M-SC,1.94,87554.0,-13223.0,13317.0,0.75,-13.12,15.21,0.09,245.0,-0.99,0.62,15.03,OX40N,NTT,DURABLES
33,ICICIGI,2252.93,-13.14,59.0,X-MC,5.87,149495.0,13362.0,17222.0,0.96,9.82,11.52,22.47,91.0,0.78,1.05,24.07,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-12.67,32.0,X-MC,2.91,86355.0,-4323.0,19231.0,-2.27,-4.77,22.27,16.44,101.0,-0.22,0.61,14.45,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,3033.0,21.43,67.0,X-SC,6.09,214545.0,23539.0,34156.0,5.22,12.32,15.92,30.21,157.0,0.69,1.51,35.78,X40N,NTT,FINANCE
84,VOLTAS,1530.0,-5.55,36.0,X-MC,4.40,198225.0,6483.0,31280.0,2.25,3.38,15.78,19.69,99.0,0.21,1.40,10.41,XY25,NTT,AC
61,SAIL,228.0,53.97,65.0,M-MC,10.42,242802.0,17840.0,149809.0,2.24,7.93,61.70,74.53,192.0,0.12,1.71,42.74,XY24,BTT,STEEL
79,UNIONBANK,163.0,-3.88,68.0,M-LC,11.83,174302.0,33462.0,11033.0,2.10,23.76,6.33,31.59,66.0,3.03,1.23,56.54,XY24,NTT,BANKS
15,CAMPUS,393.0,-24.34,48.0,M-SC,4.72,157499.0,-10276.0,66118.0,2.03,-6.13,41.98,33.28,210.0,-0.16,1.11,22.07,XY24,NTT,FOOTWEAR


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.0,-42.40,34.0,H-LC,4.89,240792.0,-20827.0,140045.0,-4.05,-7.96,58.16,45.57,7.0,-0.15,1.70,2.46,AR,NTT,ELECTRICAL
71,SYMPHONY,1306.0,-44.56,43.0,M-SC,16.44,118083.0,-53058.0,53007.0,-3.98,-31.00,44.89,-0.03,196.0,-1.00,0.83,2.95,OX40N,NTT,DURABLES
70,SURYODAY,240.0,60.09,44.0,H-SC,15.07,146146.0,-32925.0,98649.0,-3.93,-18.39,67.50,36.71,135.0,-0.33,1.03,45.01,XR,NTT,BANKS
17,CERA,9475.0,-24.34,42.0,H-SC,1.85,137931.0,-37972.0,80000.0,-3.03,-21.59,58.00,23.89,149.0,-0.47,0.97,19.12,OX40N,NTT,CERAMICS
13,BERGEPAINT,680.0,-20.05,44.0,X-MC,1.16,215075.0,-12264.0,60329.0,-2.46,-5.39,28.05,21.14,106.0,-0.20,1.52,21.23,XY24,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,26.6,48.0,M-LC,9.81,206408.0,1332.0,111233.0,1.56,0.65,53.89,54.89,52.0,0.01,1.46,24.9,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,103.20,53.0,M-SC,1.94,87554.0,-13223.0,13317.0,0.75,-13.12,15.21,0.09,245.0,-0.99,0.62,15.03,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,45.0,H-SC,2.75,221688.0,-47979.0,84308.0,-0.75,-17.79,38.03,13.47,138.0,-0.57,1.56,12.72,XY24,NTT,PAINTS
17,CERA,9475.0,-24.34,42.0,H-SC,1.85,137931.0,-37972.0,80000.0,-3.03,-21.59,58.00,23.89,149.0,-0.47,0.97,19.12,OX40N,NTT,CERAMICS
42,JCHAC,2282.0,19921.74,62.0,M-SC,1.51,97860.0,-29945.0,29935.0,-0.03,-23.43,30.59,-0.01,232.0,-1.00,0.69,10.81,OX40N,NTT,AC
67,SIS,528.0,2001.55,43.0,H-SC,2.51,84813.0,-26219.0,49828.0,-2.00,-23.61,58.75,21.26,156.0,-0.53,0.60,14.59,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-3.88,68.0,M-LC,11.83,174302.0,33462.0,11033.0,2.1,23.76,6.33,31.59,66.0,3.03,1.23,56.54,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,53.97,65.0,M-MC,10.42,242802.0,17840.0,149809.0,2.24,7.93,61.70,74.53,192.0,0.12,1.71,42.74,XY24,BTT,STEEL
31,HINDZINC,730.22,26.60,48.0,M-LC,9.81,206408.0,1332.0,111233.0,1.56,0.65,53.89,54.89,52.0,0.01,1.46,24.90,X5K,ATH,METALS
46,KPIGREEN,731.05,6.33,52.0,H-SC,9.18,126806.0,1509.0,57418.0,-1.48,1.20,45.28,47.03,141.0,0.03,0.89,58.77,MH,ATH,POWER
79,UNIONBANK,163.00,-3.88,68.0,M-LC,11.83,174302.0,33462.0,11033.0,2.10,23.76,6.33,31.59,66.0,3.03,1.23,56.54,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-40.54,52.0,M-SC,6.23,123280.0,8482.0,85556.0,-0.76,7.39,69.40,81.92,223.0,0.10,0.87,45.94,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-40.54,52.0,M-SC,6.23,123280.0,8482.0,85556.0,-0.76,7.39,69.40,81.92,223.0,0.10,0.87,45.94,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.52,52.0,H-SC,5.40,129718.0,6382.0,125243.0,-1.03,5.17,96.55,106.72,119.0,0.05,0.92,29.76,AR,ATH,MISC
46,KPIGREEN,731.05,6.33,52.0,H-SC,9.18,126806.0,1509.0,57418.0,-1.48,1.20,45.28,47.03,141.0,0.03,0.89,58.77,MH,ATH,POWER
31,HINDZINC,730.22,26.60,48.0,M-LC,9.81,206408.0,1332.0,111233.0,1.56,0.65,53.89,54.89,52.0,0.01,1.46,24.90,X5K,ATH,METALS
86,WIPRO,420.00,-16.52,38.0,M-LC,5.43,146624.0,-4321.0,113780.0,-1.48,-2.86,77.60,72.51,53.0,-0.04,1.03,2.99,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,22.0,X-LC,34.23,147269.0,-129254.0,239327.0,-0.53,-46.74,162.51,39.81,54.0,-0.54,1.04,3.80,XY24,NTT,AUTO
23,DMART,5391.45,-21.31,25.0,X-LC,7.54,200550.0,-9857.0,69029.0,-1.74,-4.68,34.42,28.12,38.0,-0.14,1.42,17.83,X40N,ATH,FMCG
44,JSWINFRA,342.00,-24.57,25.0,X-MC,4.53,189135.0,-11212.0,41723.0,-1.30,-5.60,22.06,15.23,178.0,-0.27,1.33,24.85,X40N,NTT,REALTY
12,BATAINDIA,2096.00,-41.01,30.0,X-SC,7.73,88138.0,-40532.0,85829.0,1.07,-31.50,97.38,35.20,219.0,-0.47,0.62,1.16,X40,NTT,FOOTWEAR
30,HINDUNILVR,2922.00,-14.10,32.0,X-LC,6.41,241400.0,-15663.0,50791.0,-0.12,-6.09,21.04,13.67,24.0,-0.31,1.70,13.96,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-31.74,33.0,X-MC,0.18,197610.0,-3210.0,71001.0,0.09,-1.60,35.93,33.76,80.0,-0.05,1.39,3.52,X40,ATH,FMCG
41,ITC,452.00,-38.77,42.0,X-LC,1.08,197580.0,-2558.0,23453.0,-0.85,-1.28,11.87,10.44,4.0,-0.11,1.39,4.21,X40,NTT,FMCG
66,SIEMENS,4671.50,-5.61,37.0,H-LC,1.40,151670.0,-34425.0,81902.0,-1.62,-18.50,54.00,25.51,15.0,-0.42,1.07,11.77,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-8.87,36.0,X-MC,3.22,304185.0,-23806.0,130343.0,0.43,-7.26,42.85,32.48,92.0,-0.18,2.15,3.34,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-6.24,64.0,X-LC,3.96,245805.0,11661.0,36969.0,0.89,4.98,15.04,20.77,86.0,0.32,1.73,12.24,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,PAGEIND,51769.93,-29.60,33.0,X-MC,7.41,79480.0,-4030.0,24059.0,-0.05,-4.83,30.27,23.98,82.0,-0.17,0.56,0.71,X40,ATH,APPARELS
12,BATAINDIA,2096.00,-41.01,30.0,X-SC,7.73,88138.0,-40532.0,85829.0,1.07,-31.50,97.38,35.20,219.0,-0.47,0.62,1.16,X40,NTT,FOOTWEAR
53,QUESS,986.00,-49.86,36.0,X-SC,37.39,49612.0,-15394.0,166319.0,1.54,-23.68,335.24,232.18,198.0,-0.09,0.35,1.54,XY24,NTT,MISC
20,COLPAL,3726.84,-7.18,39.0,X-MC,7.19,216720.0,-46645.0,155973.0,-0.29,-17.71,71.97,41.51,84.0,-0.30,1.53,1.70,XY25,ATH,FMCG
1,ABB,7934.00,-42.40,34.0,H-LC,4.89,240792.0,-20827.0,140045.0,-4.05,-7.96,58.16,45.57,7.0,-0.15,1.70,2.46,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-49.86,36.0,X-SC,37.39,49612.0,-15394.0,166319.0,1.54,-23.68,335.24,232.18,198.0,-0.09,0.35,1.54,XY24,NTT,MISC
51,PAGEIND,51769.93,-29.60,33.0,X-MC,7.41,79480.0,-4030.0,24059.0,-0.05,-4.83,30.27,23.98,82.0,-0.17,0.56,0.71,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.46,41.0,X-SC,4.22,81113.0,-64047.0,139977.0,-1.36,-44.12,172.57,52.31,136.0,-0.46,0.57,8.57,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-12.67,32.0,X-MC,2.91,86355.0,-4323.0,19231.0,-2.27,-4.77,22.27,16.44,101.0,-0.22,0.61,14.45,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-41.01,30.0,X-SC,7.73,88138.0,-40532.0,85829.0,1.07,-31.50,97.38,35.20,219.0,-0.47,0.62,1.16,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.33,46.0,X-LC,9.71,281229.0,-64727.0,131418.0,-0.63,-18.71,46.73,19.28,1.0,-0.49,1.98,3.97,X40,ATH,IT
39,INFY,2275.00,-19.24,51.0,X-LC,4.98,314558.0,1670.0,170019.0,0.69,0.53,54.05,54.87,3.0,0.01,2.22,8.86,X40,BTT,IT
41,ITC,452.00,-38.77,42.0,X-LC,1.08,197580.0,-2558.0,23453.0,-0.85,-1.28,11.87,10.44,4.0,-0.11,1.39,4.21,X40,NTT,FMCG
83,VBL,671.64,-16.73,53.0,X-LC,6.07,299988.0,-15854.0,128515.0,-0.35,-5.02,42.84,35.67,5.0,-0.12,2.12,8.14,X40N,ATH,FMCG
1,ABB,7934.00,-42.40,34.0,H-LC,4.89,240792.0,-20827.0,140045.0,-4.05,-7.96,58.16,45.57,7.0,-0.15,1.70,2.46,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-49.70,35.0,L-SC,27.52,74861.0,-38688.0,78739.0,-0.56,-34.07,105.18,35.27,268.0,-0.49,0.53,83.22,XR,NTT,HOTELS
7,ASIANTILES,137.00,7350.00,63.0,L-SC,12.30,81768.0,-12042.0,88661.0,-1.98,-12.84,108.43,81.67,269.0,-0.14,0.58,58.35,XR,NTT,CERAMICS
49,MASFIN,398.61,-18.70,50.0,H-SC,8.35,92205.0,-5775.0,27376.0,1.04,-5.89,29.69,22.05,152.0,-0.21,0.65,34.03,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-40.54,52.0,M-SC,6.23,123280.0,8482.0,85556.0,-0.76,7.39,69.40,81.92,223.0,0.10,0.87,45.94,XR,NTT,DURABLES
46,KPIGREEN,731.05,6.33,52.0,H-SC,9.18,126806.0,1509.0,57418.0,-1.48,1.20,45.28,47.03,141.0,0.03,0.89,58.77,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,73.00,64.0,H-SC,3.98,149939.0,-32836.0,143911.0,-0.04,-17.97,95.98,60.77,125.0,-0.23,1.06,35.03,XR,NTT,JEWELLERY
68,SONACOMS,806.63,-31.01,70.0,M-SC,6.43,85942.0,-15217.0,55218.0,1.70,-15.04,64.25,39.54,202.0,-0.28,0.61,21.19,AR,ATH,AUTO
46,KPIGREEN,731.05,6.33,52.0,H-SC,9.18,126806.0,1509.0,57418.0,-1.48,1.20,45.28,47.03,141.0,0.03,0.89,58.77,MH,ATH,POWER
5,ANGELONE,3033.00,21.43,67.0,X-SC,6.09,214545.0,23539.0,34156.0,5.22,12.32,15.92,30.21,157.0,0.69,1.51,35.78,X40N,NTT,FINANCE
7,ASIANTILES,137.00,7350.00,63.0,L-SC,12.30,81768.0,-12042.0,88661.0,-1.98,-12.84,108.43,81.67,269.0,-0.14,0.58,58.35,XR,NTT,CERAMICS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.70
1,25,44.38
2,50,75.48


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
SC,43.87
LC,33.09
MC,23.02


In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Criteria,
XY24,30.02
X40,16.25
X40N,12.48
XY25,11.92
XR,9.85
AR,7.79
OX40N,6.51
X200,1.77
X5K,1.46


In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
H-SC,24.20
X-LC,22.43
X-MC,18.68
M-SC,12.64
X-SC,5.58
M-LC,5.14
H-LC,4.54
H-MC,2.31
M-MC,1.71


In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.22,-4.79,38.67
IT,12.43,-19.32,83.24
FINANCE,12.23,-9.20,58.69
MISC,7.80,-17.48,76.74
ELECTRICAL,5.68,-12.27,52.08
PAINTS,5.58,-16.69,34.08
BANKS,4.03,-18.15,86.89
INSURANCE,3.91,0.24,36.95
AUTO,2.72,-49.39,111.74


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3317522.0,22
XR,1259892.0,13
AR,1184879.0,9
X40,851306.0,12
X40N,699106.0,9
OX40N,562395.0,9
XY25,484739.0,8
SR,274615.0,2
X5K,111233.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3401745.0,24
M-SC,1253562.0,15
X-MC,1224727.0,14
X-LC,1099519.0,13
X-SC,556944.0,6
H-LC,295505.0,3
M-LC,280626.0,4
H-MC,259783.0,2
L-SC,257024.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1192250.0      6
           AR         839953.0      5
           XR         786723.0      7
M-SC       XY24       722866.0      6
X-MC       XY24       576353.0      4
X-LC       X40        464153.0      5
           XY24       314745.0      2
X-SC       X40N       304796.0      4
X-MC       X40        301324.0      6
H-SC       SR         274615.0      2
           OX40N      250786.0      3
X-LC       X40N       234513.0      3
H-LC       AR         221947.0      2
X-MC       XY25       187253.0      2
H-MC       XY24       184147.0      1
L-SC       XR         167400.0      2
X-SC       XY24       166319.0      1
X-MC       X40N       159797.0      2
M-MC       XY24       149809.0      1
M-SC       OX40N      146349.0      4
           XR         134831.0      2
           XY25       126537.0      1
           AR         122979.0      2
M-LC       XR         113780.0      1
           X5K        111233.0      1
L-SC       OX40N       89624.0      1
X-LC       XY25        86108.0      3
X-SC       X40         85829.0      1
H-MC       OX40N       75636.0      1
H-LC       X200        73558.0      1
H-SC       MH          57418.0      1
L-MC       XR          57158.0      1
M-LC       XY25        44580.0      1
L-LC       XY25        40261.0      1
M-LC       XY24        11033.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 36.0 seconds
